In [1]:
import wrn_mixup_model
from data.datamgr import SimpleDataManager , SetDataManager
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib qt

In [2]:
num_classes = 200
image_size = 80

bridge_input_size = 640
bridge_output_size = 60
base_file = 'filelists/miniImagenet/base.json'

batch_size = 250
alpha = 2.0

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
base_datamgr    = SimpleDataManager(image_size, batch_size = batch_size)
base_loader     = base_datamgr.get_data_loader( base_file , aug = False )

In [5]:
cnn_model_path = "../checkpoints/train_64_val_16_test_20/30.tar"

cnn_model = wrn_mixup_model.wrn28_10(num_classes=200).to(device)

checkpoint = torch.load(cnn_model_path, weights_only=False)

cnn_model.load_state_dict(checkpoint['state'])

for name, param in cnn_model.named_parameters():
	param.requires_grad = False

/home/satvik/Desktop/project/integration/backbone.py:29: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(self.L, 'weight', dim=0) #split the weight update component to direction and norm


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 5.79 GiB of which 3.06 MiB is free. Process 581631 has 5.67 GiB memory in use. Including non-PyTorch memory, this process has 110.00 MiB memory in use. Of the allocated memory 1.87 MiB is allocated by PyTorch, and 130.00 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
projection_matrix = []
projection_matrix_file = os.path.join(os.getcwd(), 'projections.pt')
if not os.path.exists(projection_matrix_file):
	for batch_idx, (input_var, target_var) in enumerate(base_loader):
		input_var, target_var = input_var.to(device), target_var.to(device)
		lam = np.random.beta(alpha, alpha)
		out , _ , _ , _  = cnn_model(input_var, target_var, mixup_hidden= True, mixup_alpha = alpha , lam = lam)
		# print(out.size())
		# print(type(out))
		projection_matrix.extend(out)
	torch.save(projection_matrix, projection_matrix_file)
else:
	projection_matrix = torch.load(projection_matrix_file)

In [ ]:
len(projection_matrix)

4800

In [ ]:
projection_matrix = torch.stack(projection_matrix)

In [ ]:
U, S, Vh = torch.svd(projection_matrix, compute_uv=False, some=False)

In [ ]:
print(S.cpu().numpy())

[23.681843   13.27693    11.864118   11.282054    9.433528    9.13283
  8.779941    8.008631    7.7744675   6.825226    6.019734    5.5892816
  5.32506     5.2381105   5.019699    4.903561    4.7336783   4.4770136
  4.2729383   4.2468452   4.1121492   4.046312    3.9700103   3.7977247
  3.7576694   3.65765     3.5672007   3.527691    3.472006    3.3690312
  3.3193824   3.2839599   3.145639    3.0596638   3.0072079   2.9635873
  2.9530938   2.8601274   2.8297403   2.7840686   2.7400956   2.730636
  2.691019    2.6495037   2.6290226   2.5753536   2.5210783   2.5138235
  2.4580092   2.39507     2.3437877   2.3300946   2.2543514   2.2497795
  2.2472925   2.2205276   2.204132    2.1437607   2.1270826   2.0946105
  2.0871513   2.052187    2.048587    2.022262    1.9896657   1.9859841
  1.9564862   1.9262658   1.8970349   1.8916609   1.8804338   1.8518564
  1.8359301   1.7973727   1.7855167   1.7669227   1.7557348   1.7347474
  1.7124219   1.6948256   1.6768793   1.6525406   1.6370735   1.624

In [ ]:
x_axis = []
for i in range(S.size()[0]):
	x_axis.append(i)

In [ ]:
plt.plot(x_axis, S.tolist())
plt.show()

MESA-LOADER: failed to open radeonsi: /home/satvik/.conda/envs/final_project/bin/../lib/libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /usr/lib64/dri/radeonsi_dri.so) (search paths /usr/lib64/dri, suffix _dri)
failed to load driver: radeonsi
MESA-LOADER: failed to open swrast: /home/satvik/.conda/envs/final_project/bin/../lib/libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /usr/lib64/dri/swrast_dri.so) (search paths /usr/lib64/dri, suffix _dri)


: 